In [1]:
import os
import random
import numpy as np
import open3d as o3d
import torch.utils.data as data
import torch
import torch.optim as Optim
from torch.utils.data.dataloader import DataLoader

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
torch.backends.cudnn.benchmark = True
num_workers = 8
batch_size = 32
device="cuda:0"
LR=0.0001
epochs=100


In [3]:
train_path_partial=rf"/home/ananthakrishnak/pe/pcn-pytorch/data/PCN/train/partial/02958343"
train_path_complete=rf"/home/ananthakrishnak/pe/pcn-pytorch/data/PCN/train/complete/02958343"

valid_path_partial=rf"/home/ananthakrishnak/pe/pcn-pytorch/data/PCN/valid/partial/02958343"
valid_path_complete=rf"/home/ananthakrishnak/pe/pcn-pytorch/data/PCN/valid/complete/02958343"

test_path_partial=rf"/home/ananthakrishnak/pe/pcn-pytorch/data/PCN/test/partial/02958343"
test_path_complete=rf"/home/ananthakrishnak/pe/pcn-pytorch/data/PCN/test/complete/02958343"


In [4]:
class shapenet_data_loader(data.Dataset):
    def __init__(self, x_path, y_path, split):
        self.x_path = x_path
        self.y_path = y_path
        self.split = split
        self.partial_paths, self.complete_paths = self._load_data()

    def _load_data(self):
        lines=os.listdir(self.y_path)
        partial_paths, complete_paths = list(), list()
        for model_id in lines:
            if self.split == 'train':
                a=model_id.split('.ply')[0]
                partial_paths.append(os.path.join(self.x_path, a + '_{}.ply'))
            else:
                partial_paths.append(os.path.join(self.x_path, model_id))
            complete_paths.append(os.path.join(self.y_path, model_id ))
        return partial_paths, complete_paths
    
    def read_point_cloud(self, path):
        pc = o3d.io.read_point_cloud(path)
        return np.array(pc.points, np.float32)
    
    def random_sample(self, pc, n):
        if pc.shape[0] == 0:
            raise ValueError("Point cloud is empty, cannot sample points.")
        idx = np.random.permutation(pc.shape[0])
        if idx.shape[0] < n:
            idx = np.concatenate([idx, np.random.randint(pc.shape[0], size=n-pc.shape[0])])
        return pc[idx[:n]]
    
    def __getitem__(self, index):
        if self.split == 'train':
            partial_path = self.partial_paths[index].format(random.randint(0, 7))
        else:
            partial_path = self.partial_paths[index]
        complete_path = self.complete_paths[index]
        partial_pc = self.random_sample(self.read_point_cloud(partial_path), 2048)
        complete_pc = self.random_sample(self.read_point_cloud(complete_path), 16384)
        return torch.from_numpy(partial_pc), torch.from_numpy(complete_pc)

    def __len__(self):
        return len(self.complete_paths)

In [5]:
train_dataset = shapenet_data_loader(train_path_partial,train_path_complete,'train')
valid_dataset = shapenet_data_loader(valid_path_partial,valid_path_complete,'valid')

In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [7]:
for partial_pc, complete_pc in train_dataloader:
    print("Partial point cloud shape:", partial_pc.shape)   
    print("Complete point cloud shape:", complete_pc.shape) 
    print("Partial point cloud data:\n", partial_pc)        
    print("Complete point cloud data:\n", complete_pc)
    break  

Partial point cloud shape: torch.Size([32, 2048, 3])
Complete point cloud shape: torch.Size([32, 16384, 3])
Partial point cloud data:
 tensor([[[-0.4167,  0.0941, -0.1277],
         [ 0.4081, -0.0246,  0.0041],
         [-0.2806,  0.0650, -0.0483],
         ...,
         [-0.1093,  0.0993,  0.0324],
         [ 0.2152,  0.0170, -0.1987],
         [ 0.0035,  0.0885, -0.0581]],

        [[-0.0944,  0.0375, -0.1682],
         [-0.1593,  0.0239, -0.1821],
         [ 0.1941,  0.0355, -0.0192],
         ...,
         [-0.1544,  0.1119, -0.0133],
         [-0.2323,  0.0969, -0.0133],
         [-0.0539,  0.0860, -0.1232]],

        [[ 0.0284, -0.0261, -0.0910],
         [ 0.2929, -0.0181, -0.1575],
         [-0.3045, -0.0028, -0.1551],
         ...,
         [-0.4459,  0.0548, -0.0710],
         [-0.0527,  0.0199, -0.0837],
         [-0.3080,  0.0521, -0.0008]],

        ...,

        [[ 0.0650,  0.0217, -0.1716],
         [-0.0277,  0.1708, -0.1368],
         [ 0.1003,  0.0872, -0.1915],
     

In [8]:
import torch
import torch.nn as nn

In [9]:
class PCN(nn.Module):
    def __init__(self, num_dense=16384, latent_dim=1024, grid_size=4):
        super().__init__()
        self.num_dense = num_dense
        self.latent_dim = latent_dim
        self.grid_size = grid_size
        assert self.num_dense % self.grid_size ** 2 == 0
        self.num_coarse = self.num_dense // (self.grid_size ** 2)
        self.first_conv = nn.Sequential(nn.Conv1d(3, 128, 1),nn.BatchNorm1d(128),nn.ReLU(inplace=True),nn.Conv1d(128, 256, 1))
        self.second_conv = nn.Sequential(nn.Conv1d(512, 512, 1),nn.BatchNorm1d(512),nn.ReLU(inplace=True),nn.Conv1d(512, self.latent_dim, 1))
        self.mlp = nn.Sequential(
            nn.Linear(self.latent_dim, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 3 * self.num_coarse)
        )

        self.final_conv = nn.Sequential(
            nn.Conv1d(1024 + 3 + 2, 512, 1),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Conv1d(512, 512, 1),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Conv1d(512, 3, 1)
        )
        a = torch.linspace(-0.05, 0.05, steps=self.grid_size, dtype=torch.float).view(1, self.grid_size).expand(self.grid_size, self.grid_size).reshape(1, -1)
        b = torch.linspace(-0.05, 0.05, steps=self.grid_size, dtype=torch.float).view(self.grid_size, 1).expand(self.grid_size, self.grid_size).reshape(1, -1)
        
        self.folding_seed = torch.cat([a, b], dim=0).view(1, 2, self.grid_size ** 2).cuda()  # (1, 2, S)

    def forward(self, xyz):
        B, N, _ = xyz.shape
        
        # encoder
        feature = self.first_conv(xyz.transpose(2, 1))                                       # (B,  256, N)
        feature_global = torch.max(feature, dim=2, keepdim=True)[0]                          # (B,  256, 1)
        feature = torch.cat([feature_global.expand(-1, -1, N), feature], dim=1)              # (B,  512, N)
        feature = self.second_conv(feature)                                                  # (B, 1024, N)
        feature_global = torch.max(feature,dim=2,keepdim=False)[0]                           # (B, 1024)
        
        # decoder
        coarse = self.mlp(feature_global).reshape(-1, self.num_coarse, 3)                    # (B, num_coarse, 3), coarse point cloud
        point_feat = coarse.unsqueeze(2).expand(-1, -1, self.grid_size ** 2, -1)             # (B, num_coarse, S, 3)
        point_feat = point_feat.reshape(-1, self.num_dense, 3).transpose(2, 1)               # (B, 3, num_fine)

        seed = self.folding_seed.unsqueeze(2).expand(B, -1, self.num_coarse, -1)             # (B, 2, num_coarse, S)
        seed = seed.reshape(B, -1, self.num_dense)                                           # (B, 2, num_fine)

        feature_global = feature_global.unsqueeze(2).expand(-1, -1, self.num_dense)          # (B, 1024, num_fine)
        feat = torch.cat([feature_global, seed, point_feat], dim=1)                          # (B, 1024+2+3, num_fine)
    
        fine = self.final_conv(feat) + point_feat                                            # (B, 3, num_fine), fine point cloud

        return coarse.contiguous(), fine.transpose(1, 2).contiguous()


In [10]:
model = PCN(num_dense=16384, latent_dim=1024, grid_size=4).to(device)
optimizer = Optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.999))
lr_schedual = Optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.7)
step = len(train_dataloader) // 50

In [11]:
best_cd_l1 = 1e8
best_epoch_l1 = -1
train_step, val_step = 0, 0

In [12]:
from torch.autograd import Function

from torch.utils.cpp_extension import load

# cudapath = rf''
chamfer_3D = load(name="chamfer_3D",
sources=[rf"/home/ananthakrishnak/pe/pcn-our/chamfer_cuda.cpp",
         rf"/home/ananthakrishnak/pe/pcn-our/chamfer3D.cu",
        ])

class chamfer_3DFunction(Function):
    @staticmethod
    def forward(ctx, xyz1, xyz2):
        """
        xyz1: (B, N, 3)
        xyz2: (B, M, 3)
        """
        batchsize, n, _ = xyz1.size()
        _, m, _ = xyz2.size()
        device = xyz1.device

        dist1 = torch.zeros(batchsize, n)
        dist2 = torch.zeros(batchsize, m)

        idx1 = torch.zeros(batchsize, n).type(torch.IntTensor)
        idx2 = torch.zeros(batchsize, m).type(torch.IntTensor)

        dist1 = dist1.to(device)
        dist2 = dist2.to(device)
        idx1 = idx1.to(device)
        idx2 = idx2.to(device)
        torch.cuda.set_device(device)

        chamfer_3D.forward(xyz1, xyz2, dist1, dist2, idx1, idx2)
        ctx.save_for_backward(xyz1, xyz2, idx1, idx2)
        return dist1, dist2, idx1, idx2

    @staticmethod
    def backward(ctx, graddist1, graddist2, gradidx1, gradidx2):
        xyz1, xyz2, idx1, idx2 = ctx.saved_tensors
        graddist1 = graddist1.contiguous()
        graddist2 = graddist2.contiguous()
        device = graddist1.device

        gradxyz1 = torch.zeros(xyz1.size())
        gradxyz2 = torch.zeros(xyz2.size())

        gradxyz1 = gradxyz1.to(device)
        gradxyz2 = gradxyz2.to(device)
        chamfer_3D.backward(
            xyz1, xyz2, gradxyz1, gradxyz2, graddist1, graddist2, idx1, idx2
        )
        return gradxyz1, gradxyz2

class ChamferDistance(nn.Module):
    def __init__(self):
        super(ChamferDistance, self).__init__()

    def forward(self, input1, input2):
        """
        input1: (B, N, 3)
        input2: (B, M, 3)
        """
        dist1, dist2, _, _ = chamfer_3DFunction.apply(input1, input2)
        return dist1, dist2
    
CD = ChamferDistance()

def cd_loss_L1(pcs1, pcs2):
    """
    L1 Chamfer Distance.

    Args:
        pcs1 (torch.tensor): (B, N, 3)
        pcs2 (torch.tensor): (B, M, 3)
    """
    dist1, dist2 = CD(pcs1, pcs2)
    dist1 = torch.sqrt(dist1)
    dist2 = torch.sqrt(dist2)
    return (torch.mean(dist1) + torch.mean(dist2)) / 2.0


def l1_cd(pcs1, pcs2):
    dist1, dist2 = CD(pcs1, pcs2)
    dist1 = torch.mean(torch.sqrt(dist1), 1)
    dist2 = torch.mean(torch.sqrt(dist2), 1)
    return torch.sum(dist1 + dist2) / 2

/home/ananthakrishnak/miniconda3/envs/pcn/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [ ]:
for epoch in range(1, epochs + 1):
    # hyperparameter alpha
    if train_step < 10000:
        alpha = 0.01
    elif train_step < 20000:
        alpha = 0.1
    elif train_step < 50000:
        alpha = 0.5
    else:
        alpha = 1.0
    
    model.train()

    for i, (p, c) in enumerate(train_dataloader):
        p, c = p.to(device), c.to(device)
        optimizer.zero_grad()
        coarse_pred, dense_pred = model(p)
        loss1 = cd_loss_L1(coarse_pred, c)
        loss2 = cd_loss_L1(dense_pred, c)
        loss = loss1 + alpha * loss2
        loss.backward()
        optimizer.step()
        if (i + 1) % step == 0:
            print("Training Epoch [{:03d}/{:03d}] - Iteration [{:03d}/{:03d}]: coarse loss = {:.6f}, dense l1 cd = {:.6f}, total loss = {:.6f}".format(epoch, epochs, i + 1, len(train_dataloader), loss1.item() * 1e3, loss2.item() * 1e3, loss.item() * 1e3))
        train_step += 1
    lr_schedual.step()

    # evaluation
    model.eval()
    total_cd_l1 = 0.0
    with torch.no_grad():
        rand_iter = random.randint(0, len(val_dataloader) - 1)  # for visualization

        for i, (p, c) in enumerate(val_dataloader):
            p, c = p.to(device), c.to(device)
            coarse_pred, dense_pred = model(p)
            total_cd_l1 += l1_cd(dense_pred, c).item()

            # save into image
            # if rand_iter == i:
            #     index = random.randint(0, dense_pred.shape[0] - 1)
            #     plot_pcd_one_view(os.path.join(epochs_dir, 'epoch_{:03d}.png'.format(epoch)),
            #                         [p[index].detach().cpu().numpy(), coarse_pred[index].detach().cpu().numpy(), dense_pred[index].detach().cpu().numpy(), c[index].detach().cpu().numpy()],
            #                         ['Input', 'Coarse', 'Dense', 'Ground Truth'], xlim=(-0.35, 0.35), ylim=(-0.35, 0.35), zlim=(-0.35, 0.35))
        
        total_cd_l1 /= len(valid_dataset)
        val_step += 1
    if total_cd_l1 < best_cd_l1:
        best_epoch_l1 = epoch
        best_cd_l1 = total_cd_l1
        torch.save(model.state_dict(), os.path.join(rf"/home/ananthakrishnak/pe/pcn-our/checkpoints", 'best_l1_cd.pth'))


    


Training Epoch [001/100] - Iteration [003/178]: coarse loss = 38.733978, dense l1 cd = 277.779400, total loss = 41.511770
Training Epoch [001/100] - Iteration [006/178]: coarse loss = 28.575195, dense l1 cd = 299.597621, total loss = 31.571172
Training Epoch [001/100] - Iteration [009/178]: coarse loss = 27.415337, dense l1 cd = 186.357304, total loss = 29.278910
Training Epoch [001/100] - Iteration [012/178]: coarse loss = 24.528857, dense l1 cd = 171.646833, total loss = 26.245326
Training Epoch [001/100] - Iteration [015/178]: coarse loss = 22.349451, dense l1 cd = 158.647418, total loss = 23.935925
Training Epoch [001/100] - Iteration [018/178]: coarse loss = 22.402264, dense l1 cd = 228.538513, total loss = 24.687650
Training Epoch [001/100] - Iteration [021/178]: coarse loss = 22.459216, dense l1 cd = 179.381251, total loss = 24.253029
Training Epoch [001/100] - Iteration [024/178]: coarse loss = 21.353427, dense l1 cd = 158.186585, total loss = 22.935294
Training Epoch [001/100]